# MIPT Tutorial - Measurement-Induced Phase Transition

This tutorial demonstrates the **Measurement-Induced Phase Transition (MIPT)** in a 1D quantum circuit with random unitary gates and projective measurements.

## What is MIPT?

The MIPT arises from a competition between two processes:

1. **Unitary evolution** (Haar random gates): Creates entanglement between qubits
2. **Projective measurements** (Z-basis): Destroys entanglement locally

Depending on the measurement rate, the system exhibits two distinct phases:

- **Volume-law phase** (p < p_c ≈ 0.16): Entanglement entropy scales with system size S ~ L
- **Area-law phase** (p > p_c ≈ 0.16): Entanglement entropy remains constant S ~ const
- **Critical point** (p ≈ p_c): Logarithmic scaling S ~ log(L)

## Measurement vs Reset: A Critical Distinction


- **Measurement(:Z)**: Pure projective measurement - qubit collapses to |0⟩ or |1⟩ and stays there

## Circuit Structure

Each timestep consists of three operations:

1. `Bricklayer(:odd)` - Haar random gates on pairs (1,2), (3,4), (5,6), ...
2. `Bricklayer(:even)` - Haar random gates on pairs (2,3), (4,5), (L,1), ...
3. Z-measurements - Each site measured with probability p using `Measurement(:Z)`

This Bricklayer pattern ensures all qubits are entangled before measurements occur.

## Setup

First, activate the project environment and import QuantumCircuitsMPS:

In [ ]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using QuantumCircuitsMPS
using Printf

## Section 1: Setup and Parameters

Define the system parameters for the MIPT simulation:

In [ ]:
# Define system parameters
const L = 4                   # System size (number of qubits)
const bc = :periodic           # Boundary conditions
const n_steps = 50             # Total timesteps for simulation (used in simulate!(n_circuits=n_steps))
const p = 0.15                 # Measurement probability (near critical p_c ≈ 0.16)
const cut = L ÷ 2              # Entanglement cut position

println("=" ^ 70)
println("MIPT Example - Measurement-Induced Phase Transition")
println("=" ^ 70)
println()
println("Parameters:")
println("  L = $L (system size)")
println("  bc = $bc (boundary conditions)")
println("  n_steps = $n_steps (circuit timesteps)")
println("  p = $p (measurement probability)")
println("  cut = $cut (entanglement cut position)")
println()

## Section 2: Building the MIPT Circuit

The circuit implements the standard MIPT protocol with Bricklayer unitary gates followed by stochastic measurements.

**Key API Features**:
- `apply!(c, gate, geometry)` - Apply deterministic gates
- `apply_with_prob!(c; rng=:ctrl, outcomes=[...])` - Apply stochastic operations
- `Bricklayer(:odd)` and `Bricklayer(:even)` - Alternating brick patterns for full connectivity

In [ ]:
println("Building MIPT circuit...")
println()

# Build circuit with n_steps=1 (one timestep per circuit execution)
# We'll run this circuit n_steps times using simulate!(n_circuits=n_steps)
circuit = Circuit(L=L, bc=bc, n_steps=1) do c
    # Apply Haar random unitaries in bricklayer pattern
    apply!(c, HaarRandom(), Bricklayer(:odd))
    apply!(c, HaarRandom(), Bricklayer(:even))
    
    # Apply projective Z-measurements with probability p
    apply_with_prob!(c; rng=:ctrl, outcomes=[
        (probability=p, gate=Measurement(:Z), geometry=AllSites())
    ])
end

println("✓ Circuit built successfully")
println("  Circuit n_steps: $(circuit.n_steps) (timesteps per circuit run)")
println("  System size: $(circuit.L) qubits")
println("  Boundary conditions: $(circuit.bc)")
println()

## Section 3: Simulation with Entanglement Tracking

We track the **entanglement entropy** at the central cut (position L÷2) to monitor the competition between:
- Unitary entanglement generation
- Measurement-induced disentanglement

The `EntanglementEntropy` observable computes the von Neumann entropy across a bipartite cut:

$$S = -\sum_i \lambda_i^2 \log(\lambda_i^2)$$

where $\lambda_i$ are the Schmidt coefficients from the SVD of the MPS bond.

In [ ]:
println("Running simulation...")
println()

# Create simulation state with RNG registry
state = SimulationState(
    L=L,
    bc=bc,
    rng=RNGRegistry(ctrl=42, born=1, haar=2, proj=3)
)

# Initialize to product state |0⟩⊗L
initialize!(state, ProductState(x0=0//1))

# Track entanglement entropy at the central cut
track!(state, :entropy => EntanglementEntropy(; cut=cut))

# Run simulation: execute circuit n_steps times (n_circuits=n_steps)
# The circuit has n_steps=1, so we run it n_steps times to get n_steps total timesteps
simulate!(circuit, state; n_circuits=n_steps, record_when=:every_step)

# Extract entropy values from state
entropy_vals = state.observables[:entropy]

println("✓ Simulation complete")
println("  Recorded $(length(entropy_vals)) entropy values")
println()

## Section 4: Results and Analysis

Print the entanglement entropy evolution at regular intervals to observe the dynamics:

In [ ]:
println("Entanglement Entropy Evolution:")
println("-" ^ 70)

# Print entropy at every 10 steps
for t in 1:n_steps
    if t % 10 == 0 || t == n_steps
        println("Step $t: Entanglement entropy = $(Printf.@sprintf("%.6f", entropy_vals[t]))")
    end
end

println()
println("=" ^ 70)
println("Simulation complete!")
println()
println("Physical Interpretation:")
println("  - At p=0.15 (near critical p_c ≈ 0.16), entropy shows non-trivial dynamics")
println("  - Volume-law phase (p < p_c): S ~ L (high entanglement)")
println("  - Area-law phase (p > p_c): S ~ const (low entanglement)")
println("  - Critical point (p ≈ p_c): S ~ log(L)")
println("=" ^ 70)

## Exercises

Try modifying the measurement rate `p` to explore different phases:

1. **Deep volume-law phase**: Set `p = 0.05` (low measurement rate)
   - Expected: Entropy grows and saturates at high value ~ L
   - Unitary dynamics dominate

2. **Near-critical** (current): `p = 0.15`
   - Expected: Non-trivial dynamics with intermediate entropy
   - Competition between unitary and measurement

3. **Area-law phase**: Set `p = 0.30` (high measurement rate)
   - Expected: Entropy remains low ~ const
   - Measurements suppress entanglement

**Challenge**: Plot the entanglement entropy evolution for all three values of `p` on the same graph. What differences do you observe?

**Hint**: You can access the entropy values via `state.observables[:entropy]`.